In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [12]:
path_output=os.path.join(os.getcwd(), "..", "data", "output")

In [13]:
features = pd.read_csv(os.path.join(path_output, "Features.csv"))
features = features.drop("Unnamed: 0", axis=1)

### Take Params from csv

In [14]:
params = pd.read_csv(os.path.join(os.getcwd(), "..", "data", "input", "params.csv")).drop("0", axis=1)
_test_size = params[params["param"]=="test_size"]["value"].values[0]
_random_state = int(params[params["param"]=="random_state"]["value"].values[0])
_epochs = int(params[params["param"]=="epochs"]["value"].values[0])
_validation_size = params[params["param"]=="validation_size"]["value"].values[0]

### Train SVM

In [15]:
def train_support_vector_machine(X_train_scaled, y_train, time_res, hex_size, kernel):
    """
    Train SVM

    Train and save an SVM model.
    Then evaluate the error metrics by another method.

    Args:
        X_train_scaled (DataFrame):   Scaled X input of train set (matrix)
        y_train (Series):             y output to train on (vector)
    Returns:
        svm_regression_sets (array): true y values and predicted y values for train and validation set
    """
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    # print("Splitting train test data...")
    X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, random_state=_random_state, test_size=_validation_size)

    # print("Initializing SVR...")
    svr = make_pipeline(StandardScaler(), SVR(kernel=kernel, cache_size=2000, max_iter=1000))
    # svr = SVR(kernel=kernel, max_iter=1000)
    # print(svr)
    # print("Fitting SVR...")
    svr.fit(X_train, y_train)
    print("Score (Train):", svr.score(X_train, y_train))
    
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    # print("Saving SVR model...")
    filename = "SVR_model_"+time_res+"_"+hex_size+"_"+kernel+".pkl"
    pickle.dump(svr, open(os.path.join(os.getcwd(), "..", "data", "output", "models", filename), "wb"))
    
    # print("Predicting y with train data...")
    y_prediction_train = svr.predict(X_train)
    # print("Predicting y with validation data...")
    y_prediction_val = svr.predict(X_val)
    
    # Evaluate Model
    # print(confusion_matrix(y_train,y_prediction_train_lin))
    # print(classification_report(y_train,y_prediction_train_lin))
    # print(confusion_matrix(y_val,y_prediction_val_lin))
    # print(classification_report(y_val,y_prediction_val_lin))
    
    svm_regression_sets = [y_train, y_val, y_prediction_train, y_prediction_val]
    return svm_regression_sets

### Loss visualization

In [16]:
def plot_train_loss(svm_regression_sets, time_res, hex_size, kernel):
    """
    Plot the train and validation loss of Support Vector Machine.

    Args:
        history (Object): History of loss during training of support vector machine
        time_res (str): time resolution to train on
    Returns:
        No return
    """
    print("Plot difference between Train and Predicted Train...")
    y_train = svm_regression_sets[0]
    y_val = svm_regression_sets[1]
    y_prediction_train = svm_regression_sets[2]
    y_prediction_val = svm_regression_sets[3]
    
    
    mae_train = mean_absolute_error(y_train, y_prediction_train)
    mae_val = mean_absolute_error(y_val, y_prediction_val)
    mse_train = mean_squared_error(y_train, y_prediction_train)
    mse_val = mean_squared_error(y_val, y_prediction_val)
    r2_train = r2_score(y_train, y_prediction_train)
    r2_val = r2_score(y_val, y_prediction_val)
    
    print()
    print("=== TRAIN ===")
    print("MAE:", mae_train)
    print("MSE:", mse_train)
    print("R2:", r2_train)
    print()
    
    print("=== VALIDATION ===")
    print("MAE:", mae_val)
    print("MSE:", mse_val)
    print("R2:", r2_val)
    print()
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 8), dpi=300)
    
    ax1.plot(y_prediction_train, y_train, "bo")
    ax1.set_title("Train Y vs Predicted Train Y ("+time_res+", "+hex_size+", "+kernel+")", fontsize=20)
    ax1.set_xlabel("Predicted Train Y", fontsize=18)
    ax1.set_ylabel("Train Y", fontsize=18)
    
    ax2.plot(y_prediction_val, y_val, "bo")
    ax2.set_title("Validation Y vs Predicted Validation Y ("+time_res+", "+hex_size+", "+kernel+")", fontsize=20)
    ax2.set_xlabel("Predicted Validation Y", fontsize=18)
    ax2.set_ylabel("Validation Y", fontsize=18)
    
    fig.savefig(os.path.join(path_output, "SVM_Train_Validation_Real_vs_Predicted_"+time_res+"_"+hex_size+"_"+kernel+".png"))
    plt.close(fig)
    
    
    
    # print("Plot training and visualization loss...")
    
    # Plotting the training and validation loss
    # loss = history.history["loss"]
    # val_loss = history.history["val_loss"]

    # epochs = range(1, len(loss) + 1)
    # fig, ax = plt.subplots(figsize=(16, 8), dpi=300)
    # ax.plot(epochs, loss, "bo", label="Training loss")
    # ax.plot(epochs, val_loss, "b", label="Validation loss")
    # ax.set_title("Training and validation loss "+time_res+"_"+hex_size, fontsize=18)
    # ax.set_xlabel("Epochs", fontsize=16)
    # ax.set_ylabel("Loss", fontsize=16)
    # plt.legend()
    # plt.show()
    # fig.savefig(os.path.join(path_output, "SVM_error_per_epoch_"+time_res+"_"+hex_size+".png"))
    # plt.close(fig)

### Run SVM

In [17]:
def train_SVM(time_res="24_demand", hex_size="hexa_big", kernel="linear"):
    """
    Split the data in train and test set by 0.3 test set. 
    Then Scale the data and do a PCA. 
    Last train the SVM on the chosen time resolution
    
    Args:
        time_res (str): time resolution to train on
        
    Returns:
        No return
    """
    print("Time Resolution is", time_res)
    #print("Split Data with random state", _random_state, "and test size", str(_test_size)+"...")
    # features_X = features.drop(["24_sum", "6_sum", "2_sum", "1_sum"], axis=1)
    features_X = features.drop(["24_demand", "24_demand_hex_big", "24_demand_hex_small", "24_agg_time",
                                "6_demand", "6_demand_hex_big", "6_demand_hex_small", "6_agg_time",
                                "2_demand", "2_demand_hex_big", "2_demand_hex_small", "2_agg_time",
                                "1_demand", "1_demand_hex_big", "1_demand_hex_small", "1_agg_time",
                                "24_available_hex_big"], axis=1)
    features_y = features[time_res]
    
    # Spatial Resolution
    print("Spatial Resolution is", hex_size)
    if hex_size=="hexa_small":
        features_X = features_X.drop("hexa_big", axis=1)
    else:
        features_X = features_X.drop("hexa_small", axis=1)
    
    # Kernel
    print("Kernel is", kernel)
    
    #Split
    X_train, X_test, y_train, y_test = train_test_split(features_X, features_y, random_state=_random_state, test_size=_test_size)

    #print("Scale", hex_size, "Data with Standard Scaler...")
    with open(os.path.join(path_output, "models", "Standard_Scaler_"+hex_size+".pkl"), "rb") as f:
        standard_scaler = pickle.load(f)
    X_train_scaled = standard_scaler.transform(X_train)

    print("Do PCA on", hex_size, "Data...")
    with open(os.path.join(path_output, "models", "PCA_"+hex_size+".pkl"), "rb") as f:
        pca = pickle.load(f)
    X_train_transformed = pca.transform(X_train_scaled)
    
    # PCA
    # pca = PCA(n_components=10)
    # pca.fit(X_train_scaled)
    # pca_explained_variance = pca.explained_variance_ratio_
    #  print("Var explained:", pca_explained_variance)
    # print("Sum var explained", sum(pca_explained_variance))
    # X_train_transformed = pca.transform(X_train_scaled)
    # print("X_train_transformed:", X_train_transformed)

    print("Train", "SVM_Regression_Model_"+time_res+"_"+hex_size+"_"+kernel, "...")
    svm_regression_sets = train_support_vector_machine(X_train_transformed, y_train.to_numpy(), time_res=time_res, hex_size=hex_size, kernel=kernel)
    plot_train_loss(svm_regression_sets, time_res=time_res, hex_size=hex_size, kernel=kernel)

In [18]:
#Train the SVM for each time resolution.
hex_size = ["hexa_big", "hexa_small"]
time_resolutions = ["24_demand", "1_demand"]
kernels = ["linear", "rbf", "poly", "sigmoid"]

# """
for time in time_resolutions:
    for size in hex_size:
        for kernel in kernels:
            train_SVM(time_res=time, hex_size=size, kernel=kernel)
            print()
# """

train_SVM(time_res="24_available_hex_big", hex_size="hexa_big", kernel="linear")
# train_SVM(time_res="24_demand", hex_size="hexa_big", kernel="sigmoid")
# train_SVM(time_res="24_demand", hex_size="hexa_big", kernel="precomputed")
# train_SVM(time_res="24_sum", hex_size="hexa_big", kernel="rbf")
# train_SVM(time_res="24_sum", hex_size="hexa_big", kernel="poly")
# train_SVM(time_res="1_sum", hex_size="hexa_small", kernel="linear")
print("Done")


Time Resolution is 24_demand
Spatial Resolution is hexa_big
Kernel is linear
Do PCA on hexa_big Data...
Train SVM_Regression_Model_24_demand_hexa_big_linear ...


/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Score (Train): -0.28315578750727877
Plot difference between Train and Predicted Train...

=== TRAIN ===
MAE: 1109.5313708621782
MSE: 3007555.465012589
R2: -0.28315578750727877

=== VALIDATION ===
MAE: 1108.6876790385004
MSE: 2989228.991416422
R2: -0.2860404644788419


Time Resolution is 24_demand
Spatial Resolution is hexa_big
Kernel is rbf
Do PCA on hexa_big Data...
Train SVM_Regression_Model_24_demand_hexa_big_rbf ...


/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Score (Train): -1.9323751392781046
Plot difference between Train and Predicted Train...

=== TRAIN ===
MAE: 2524.181723707827
MSE: 6873117.794009784
R2: -1.9323751392781046

=== VALIDATION ===
MAE: 2523.585386589764
MSE: 6870064.215466413
R2: -1.9556720478853031


Time Resolution is 24_demand
Spatial Resolution is hexa_big
Kernel is poly
Do PCA on hexa_big Data...
Train SVM_Regression_Model_24_demand_hexa_big_poly ...


...

Score (Train): -458.87463420959585
Plot difference between Train and Predicted Train...

=== TRAIN ===
MAE: 7104.930862387642
MSE: 50590213.16076102
R2: -458.87463420959585

=== VALIDATION ===
MAE: 7103.208320780201
MSE: 50563078.36117588
R2: -470.1025960211533


Time Resolution is 1_demand
Spatial Resolution is hexa_small
Kernel is sigmoid
Do PCA on hexa_small Data...
Train SVM_Regression_Model_1_demand_hexa_small_sigmoid ...


/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Score (Train): -7.546711977675004
Plot difference between Train and Predicted Train...

=== TRAIN ===
MAE: 944.3860188065182
MSE: 940212.7201848303
R2: -7.546711977675004

=== VALIDATION ===
MAE: 945.0187080810281
MSE: 940941.0945573403
R2: -7.7668671828595475


Time Resolution is 24_available_hex_big
Spatial Resolution is hexa_big
Kernel is linear
Do PCA on hexa_big Data...
Train SVM_Regression_Model_24_available_hex_big_hexa_big_linear ...


/opt/conda/lib/python3.8/site-packages/sklearn/svm/_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Score (Train): -17.12848878500209
Plot difference between Train and Predicted Train...

=== TRAIN ===
MAE: 584.8133312614224
MSE: 362175.5498883341
R2: -17.12848878500209

=== VALIDATION ===
MAE: 584.9035838597853
MSE: 362293.6440177584
R2: -17.129510750473653

Done
